In [11]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import shutil
import os
import cv2
import numpy as np

# Define the function to upload and save the image
def upload_image():
    file_path = filedialog.askopenfilename()
    if file_path:
        image = Image.open(file_path)
        image.thumbnail((300, 300))  # Resize image if necessary
        photo = ImageTk.PhotoImage(image)
        image_label.config(image=photo)
        image_label.image = photo  # Keep a reference to avoid garbage collection
        save_image(file_path)
        messagebox.showinfo("Success", "Image uploaded successfully!")
 
def save_image(file_path):
    
    filename = os.path.basename(file_path)
    shutil.copy(file_path, 'input.jpg')
    print("Image saved to: input.jpg")
 
# Create the main window
root = tk.Tk()
root.title("Image Uploader")
 
# Set window size
root.geometry("500x500")
 
# Create and pack widgets
upload_button = tk.Button(root, text="Upload Image", command=upload_image)
upload_button.pack(pady=10)
 
image_label = tk.Label(root)
image_label.pack()
 
# Run the application
root.mainloop()

yolo = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")


with open("coco.names", "r") as file:
    classes = [line.strip() for line in file.readlines()]
layer_names = yolo.getLayerNames()
output_layers = [layer_names[i - 1] for i in yolo.getUnconnectedOutLayers()]

colorRed = (0,0,255)
colorGreen = (0,255,0)

# #Loading Images
name = "input.jpg"
img = cv2.imread(name)
width = 200
height =200
rimage = cv2.resize(img, (width, height))
cv2.imwrite("input.jpg",rimage)
height, width, channels = img.shape

# # Detecting objects
blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

yolo.setInput(blob)
outputs = yolo.forward(output_layers)

class_ids = []
confidences = []
boxes = []
for output in outputs:
    for detection in output:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)

            x = int(center_x - w / 2)
            y = int(center_y - h / 2)

            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
for i in range(len(boxes)):
    if i in indexes:
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        cv2.rectangle(img, (x, y), (x + w, y + h), colorGreen, 3)
        cv2.putText(img, label, (x, y + 150), cv2.FONT_HERSHEY_PLAIN, 8, colorRed, 8)


#cv2.imshow("Image", img)
width = 500
height = 500
rimage = cv2.resize(img, (width, height))
cv2.imwrite("output.jpg",rimage)

cv2.imshow("output.jpg",rimage)
cv2.waitKey(0)
cv2.destroyAllWindows()

Image saved to: input.jpg
